## Hive Popular Tags per Year
Compare most popular tags in year 2016 with tags in 2009. Tag popularity is the number of questions (post_type_id=1) with this tag. Output top 10 tags in format:
```
tag <tab> rank in 2016 <tab> rank in 2009 <tab> tag popularity in 2016 <tab> tag popularity in 2009
```

In [1]:
%%writefile query.hql

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar;

WITH 
TAG_EX_2016 AS (
    SELECT post_type_id,tag
    FROM stackoverflow_.posts LATERAL VIEW explode(tags) tagTable AS tag
    WHERE Year=2016 and tags IS NOT NULL
),
TAG_EX_2009 AS (
    SELECT post_type_id,tag
    FROM stackoverflow_.posts LATERAL VIEW explode(tags) tagTable AS tag
    WHERE Year=2009 and tags IS NOT NULL
),
TAG_COUNT_2016 AS (
    SELECT tag, COUNT(post_type_id) AS id_count_2016
    FROM TAG_EX_2016
    GROUP BY tag
),
TAG_COUNT_2009 AS (
    SELECT tag, COUNT(post_type_id) AS id_count_2009
    FROM TAG_EX_2009
    GROUP BY tag
),
TAG_RANKED_2016 AS (
    SELECT tag AS tag_2016, id_count_2016, RANK() OVER(ORDER BY id_count_2016 DESC) AS tag_rank_2016
    FROM TAG_COUNT_2016
),
TAG_RANKED_2009 AS (
    SELECT tag AS tag_2009, id_count_2009, RANK() OVER(ORDER BY id_count_2009 DESC) AS tag_rank_2009
    FROM TAG_COUNT_2009
)
SELECT tag_2016,tag_rank_2016,tag_rank_2009,id_count_2016,id_count_2009
FROM TAG_RANKED_2016,TAG_RANKED_2009
WHERE tag_2016=tag_2009
ORDER BY tag_rank_2016
LIMIT 10;

Overwriting query.hql


In [2]:
! hive --database stackoverflow_ -f query.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.147 seconds
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar]
Query ID = jovyan_20180313044242_b72c9b88-70c0-4400-abc1-bcae0821f869
Total jobs = 8
Launching Job 1 out of 8
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1520896233122_0022, Tracking URL = http://b4907